# Amazing RL Agent

In [1]:
import gymnasium as gym
import torch
import importlib

# Render screen, set to false if training
RENDER_AGENT = True


In [2]:
# Test pytorch is working

x = torch.rand(size=(1000,))
x = x + 1

# If you have a nvidia gpu with torch installed correctly
# you can move the tensors to the gpu for some extra
# speed, not necessary although.
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE}")
# When creating a new tensor make sure you move it to a
# device using .to(DEVICE), such as:
x = x.to(DEVICE)
# The tensor in x has been moved to the device


Using cpu


In [3]:
# Import agent
import agents

# Reload module as jupyter notebooks wont update to any changes youve made
importlib.reload(
    agents
)
agent = agents.Agent()


In [4]:
# Globals
steps = 200
seed = 27


# Standard Agent
env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="human")
# observation, info = env.reset(seed=seed)
# for _ in range(steps):
#     action = agent.choose_action(
#         env.action_space, observation
#     )  # this is where you would insert your policy
#     observation, reward, terminated, truncated, info = env.step(action)
#     agent.update(observation, reward, terminated, truncated)
#     if terminated or truncated:
#         observation, info = env.reset(seed=seed)
#     env.render()
# env.close()


# Rounding Temporal Difference Model
state_space = env.observation_space
action_space = env.action_space

discrete_actions = 5  # number of possible actions
discrete_states = 2  # number of states per dimension
"""
Bipedal Environment has 24 dimensions, with values of: 
discrete_actions = 5  
discrete_states = 2
we then have 2^24 * 5 = 268,435,456 state-action spaces 
"""

agent_TD = agents.RoundingTDAgent(state_space, action_space, discrete_actions, discrete_states)

observation, info = env.reset(seed=seed)
action = agent_TD.choose_action(env.action_space, observation)
for _ in range(steps):
    observation_next, reward, terminated, truncated, info = env.step(action)
    action_next = agent_TD.choose_action(env.action_space, observation_next)

    agent_TD.update(observation, observation_next, reward, terminated, truncated)

    observation = observation_next
    action = action_next

    if terminated or truncated:
        observation, info = env.reset(seed=seed)
        action = agent_TD.choose_action(env.action_space, observation)

    env.render()
env.close()
